<a href="https://colab.research.google.com/github/dandywic10/SignLanguageTranslator/blob/main/DeepLearningDandy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install dependencies
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, Javascript, HTML, clear_output
import PIL.Image
from base64 import b64decode, b64encode
import io
import time

# Setup MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Dictionary untuk gesture
gesture_dict = {
    'thumbs_up': 'Halo',
    'palm_open': 'Terima Kasih',
    'fist': 'Tolong',
    'peace': 'Selamat Tinggal',
    'pointing': 'Ya',
    'thumbs_down': 'Tidak',
    'ok_sign': 'Baik',
    'three_fingers': 'Minta Maaf',
    'pinky_up': 'Janji',
    'palm_down': 'Sudah'
}

def get_webcam():
    js = Javascript('''
    async function getWebcam() {
        const div = document.createElement('div');
        const videoContainer = document.createElement('div');
        const resultDiv = document.createElement('div');

        videoContainer.style.position = 'relative';
        resultDiv.style.position = 'absolute';
        resultDiv.style.top = '10px';
        resultDiv.style.left = '10px';
        resultDiv.style.color = 'green';
        resultDiv.style.fontSize = '24px';
        resultDiv.style.fontWeight = 'bold';
        resultDiv.id = 'resultText';

        document.body.appendChild(div);
        div.appendChild(videoContainer);
        videoContainer.appendChild(resultDiv);

        const video = document.createElement('video');
        video.style.width = '500px';
        video.autoplay = true;
        video.playsinline = true;
        videoContainer.appendChild(video);

        const canvas = document.createElement('canvas');
        canvas.style.display = 'none';
        videoContainer.appendChild(canvas);

        const stopButton = document.createElement('button');
        stopButton.textContent = 'Stop';
        stopButton.style.display = 'block';
        stopButton.style.marginTop = '10px';
        div.appendChild(stopButton);

        try {
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;

            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            // Function to process each frame
            const processFrame = () => {
                if (!video.paused && !video.ended) {
                    const context = canvas.getContext('2d');
                    context.drawImage(video, 0, 0, canvas.width, canvas.height);

                    // Send frame data for processing
                    const imageData = canvas.toDataURL('image/jpeg', 0.8);
                    google.colab.kernel.invokeFunction('process_frame', [imageData], {});

                    requestAnimationFrame(processFrame);
                }
            };

            video.onloadedmetadata = () => {
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                processFrame();
            };

            stopButton.onclick = () => {
                stream.getTracks().forEach(track => track.stop());
                div.remove();
            };

        } catch (error) {
            console.error('Error accessing webcam:', error);
        }
    }

    getWebcam();
    ''')
    display(js)

def detect_gesture(hand_landmarks):
    landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])

    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]

    def fingers_extended():
        return [
            thumb_tip[1] < landmarks[3][1],  # thumb
            index_tip[1] < landmarks[6][1],  # index
            middle_tip[1] < landmarks[10][1],  # middle
            ring_tip[1] < landmarks[14][1],  # ring
            pinky_tip[1] < landmarks[18][1]   # pinky
        ]

    extended = fingers_extended()

    # Original gestures dengan logika yang lebih akurat
    if extended[0] and not any(extended[1:]):  # Hanya jempol
        return 'thumbs_up'
    elif all(extended):  # Semua jari terangkat
        return 'palm_open'
    elif not any(extended):  # Semua jari tertutup
        return 'fist'
    elif extended[1] and extended[2] and not extended[3] and not extended[4]:  # Peace
        return 'peace'
    elif extended[1] and not extended[2] and not extended[3] and not extended[4]:  # Pointing
        return 'pointing'

    # Gesture baru
    elif not extended[0] and not any(extended[1:]):  # Jempol ke bawah
        return 'thumbs_down'
    elif distance(thumb_tip, index_tip) < 0.05 and all(extended[2:]):  # OK sign
        return 'ok_sign'
    elif extended[1] and extended[2] and extended[3] and not extended[4]:  # Three fingers
        return 'three_fingers'
    elif not any(extended[:4]) and extended[4]:  # Pinky up
        return 'pinky_up'
    elif all(landmarks[tip][1] > landmarks[tip-2][1] for tip in [8,12,16,20]) and thumb_tip[1] > landmarks[3][1]:  # Palm down
        return 'palm_down'

    return None

def distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

def process_frame(frame_data):
    try:
        # Decode frame data
        image_data = b64decode(frame_data.split(',')[1])
        nparr = np.frombuffer(image_data, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Process with MediaPipe
        rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        detected_gesture = None
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                detected_gesture = detect_gesture(hand_landmarks)
                if detected_gesture:
                    # Update result text via JavaScript
                    display(Javascript(f'''
                        document.getElementById('resultText').textContent =
                            "Terdeteksi: {gesture_dict.get(detected_gesture, 'Unknown')}";
                    '''))
                    break

        if not detected_gesture:
            display(Javascript('''
                document.getElementById('resultText').textContent = "Menunggu gesture...";
            '''))

    except Exception as e:
        print(f"Error processing frame: {str(e)}")

# Register callback function
from google.colab import output
output.register_callback('process_frame', process_frame)

# Print panduan
print("Panduan Gesture Bahasa Isyarat:")
for gesture, arti in gesture_dict.items():
    print(f"- {gesture}: {arti}")

print("\nKlik 'Allow' untuk mengizinkan akses kamera")
print("Klik tombol 'Stop' untuk menghentikan")

# Mulai webcam
get_webcam()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>